In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [2]:
#Loading datasets
df_geo = pd.read_csv('CommAreas.csv')

#Find a way to join these? Below have community areas joined onto bus_stops. bus_df has ridership numbers
bus_df = pd.read_csv('CTA_-_Ridership_-_Bus_Routes_-_Monthly_Day-Type_Averages___Totals.csv') 
bus_stops_df = pd.read_csv('CTA_BusStops.csv') 

#Emily's cleaned train data
l_station_cleaned = pd.read_csv('L_Station_Cleaned.csv')

In [ ]:
#Community areas dataset uses multipolygon geometry coordinates 


#DATASETS THAT DONT HAVE COMMUNITY AREAS AND NEED FOR ANALYSIS:
"""
police sentiment - districts and sectors # Will need to figure out coordinate data for sectors/districts to do 


train ridership - station_ids and names
bus_df - route names (stations) - find lats and lons for stations and then convert

train_stops - lat and lon, stop and station names
bus_stops - streets, point geometry coordinates

"""

In [ ]:
#STEPS TO DO TO CONVERT LOCATIONS:
"""
Convert dataframes into geodataframes.

lat and lon need to be converted into spatial formats (multipolyhon and point geometries already spatial formats)
    (I also ended up converting point and multipolygon anyway because it didn't work if I didn't. Maybe wasn't originally a float or 
    something. I don't really know.) 
    
Perform a spatial join, will put point geometries within the multipolygon geometries of community areas


"""

## Converting dataframes into geoframes and converting to spatial formats


In [3]:
#Community areas dataframe


# Convert the community areas muiltypolygon coordinates column to geometries
df_geo['geometry'] = df_geo['the_geom'].apply(wkt.loads)
df_geo = gpd.GeoDataFrame(df_geo, geometry='geometry')

# Drop the 'the_geom' column. No longer needed.
#Also drop other columns not being used
df_geo = df_geo.drop(columns=['the_geom','PERIMETER', 'AREA','COMAREA_','COMAREA_ID','AREA_NUMBE'])

In [4]:
#Train stops dataframe


# Convert lat/long to Point geometry
#Will add POINT coordinates as a column in df
l_station_cleaned['geometry'] = l_station_cleaned.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
points_geo_df = gpd.GeoDataFrame(l_station_cleaned, geometry='geometry')


In [5]:
#Bus stops dataframe 

# Convert the 'the_geom' column to Point geometries
bus_stops_df['geometry'] = bus_stops_df['the_geom'].apply(wkt.loads)

# Convert the DataFrame to a GeoDataFrame
bus_stops_df = gpd.GeoDataFrame(bus_stops_df, geometry='geometry')
#Drop previous column
bus_stops_df = bus_stops_df.drop(columns=['the_geom'])



## Performing the spatial joins


In [6]:
train_stops_converted = gpd.sjoin(points_geo_df, df_geo, how="inner", op="within")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [7]:
bus_stops_converted = gpd.sjoin(bus_stops_df, df_geo, how="inner", op="within")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


## Cleaning and organizing new data

In [8]:
#Trains data

#Remove irrelevant columns
train_stops_converted = train_stops_converted.drop(columns=['Month', 'Longitude', 'Latitude','index_right'])
#Rename columns 
train_stops_converted.rename(columns={'Month_Beginning': 'Month', 'geometry': 'Lat_Lon', 'AREA_NUM_1':'Comm_Num', 'COMMUNITY':'Community'}, inplace=True)


In [9]:
train_stops_converted.head(5)

,Year,Month,Station_ID,Station_Name,Avg_Weekday_Rides,Avg_Saturday_Rides,Avg_Sunday/Holiday_Rides,Monthly_Total,Lat_Lon,Community,Comm_Num,SHAPE_AREA,SHAPE_LEN
0,2001,2001-01-01,40450,95th/Dan Ryan,14162.6,7472.8,4914.4,366040,POINT (-87.62434 41.72238),ROSELAND,49,1.343137e+08,56632.795429
1,2001,2001-02-01,40450,95th/Dan Ryan,14445.6,7166.5,5282.8,338708,POINT (-87.62434 41.72238),ROSELAND,49,1.343137e+08,56632.795429
2,2001,2001-03-01,40450,95th/Dan Ryan,14521.9,7686.8,5239.8,378875,POINT (-87.62434 41.72238),ROSELAND,49,1.343137e+08,56632.795429
3,2001,2001-04-01,40450,95th/Dan Ryan,14106.8,7716.5,5349.4,353856,POINT (-87.62434 41.72238),ROSELAND,49,1.343137e+08,56632.795429
4,2001,2001-05-01,40450,95th/Dan Ryan,14591.5,7482.8,5357.8,377733,POINT (-87.62434 41.72238),ROSELAND,49,1.343137e+08,56632.795429


In [10]:
#Bus Data

#Remove irrelevant columns
bus_stops_converted = bus_stops_converted.drop(columns=['CITY', 'index_right'])
#Rename columns 
bus_stops_converted.rename(columns={'geometry': 'Lat_Lon', 'AREA_NUM_1':'Comm_Num', 'COMMUNITY':'Community'}, inplace=True)


In [11]:
bus_stops_converted.head(5)

,SYSTEMSTOP,STREET,CROSS_ST,DIR,POS,ROUTESSTPG,OWLROUTES,PUBLIC_NAM,Lat_Lon,Community,Comm_Num,SHAPE_AREA,SHAPE_LEN
0,15189,CICERO,BERTEAU (north leg),SB,FS,"54,54A",NaN,Cicero & Berteau,POINT (-87.74751 41.95689),PORTAGE PARK,15,1.101961e+08,46520.642138
32,11619,CENTRAL,BYRON,NB,NS,85,NaN,Central & Byron,POINT (-87.76686 41.95124),PORTAGE PARK,15,1.101961e+08,46520.642138
38,17175,MILWAUKEE,LAWRENCE,SEB,FS,56,NaN,Milwaukee & Lawrence,POINT (-87.76069 41.96772),PORTAGE PARK,15,1.101961e+08,46520.642138
173,11526,CENTRAL,CULLOM,SB,NS,85,NaN,Central & Cullom,POINT (-87.76725 41.95873),PORTAGE PARK,15,1.101961e+08,46520.642138
213,11618,CENTRAL,GRACE,NB,NS,85,NaN,Central & Grace,POINT (-87.76680 41.94944),PORTAGE PARK,15,1.101961e+08,46520.642138


# #TODO 
## - Join bus ridership data onto bus stops above (bus_df)
        - Currently trying to figure out commonality between these sets to join them
## - Upload these new datasets to database 
### - Maybe do some more cleaning (brain started to try at the end of this)

In [19]:
bus_df.head(100)

,route,routename,Month_Beginning,Avg_Weekday_Rides,Avg_Saturday_Rides,Avg_Sunday-Holiday_Rides,MonthTotal
0,1,Indiana/Hyde Park,01/01/2001,6982.6,0.0,0.0,153617
1,2,Hyde Park Express,01/01/2001,1000.0,0.0,0.0,22001
2,3,King Drive,01/01/2001,21406.5,13210.7,8725.3,567413
3,4,Cottage Grove,01/01/2001,22432.2,17994.0,10662.2,618796
4,6,Jackson Park Express,01/01/2001,18443.0,13088.2,7165.6,493926
5,7,Harrison,01/01/2001,5504.4,0.0,0.0,121097
6,8,Halsted,01/01/2001,19582.2,12420.0,8280.8,521892
7,8A,South Halsted,01/01/2001,3196.5,3006.6,1336.2,89030
8,9,Ashland,01/01/2001,29265.4,22621.7,15336.1,811006
9,10,Museum of S & I,01/01/2001,0.0,562.6,372.9,4115


In [21]:
# Check if a station exists in the 'routename' column
exists = bus_df['routename'].isin(['Central']).any()

# Print the result
print("Does route exist in the 'routename' column:", exists)


Does route exist in the 'routename' column: True


In [22]:
# Check if a station exists in the 'routename' column
exists = bus_stops_converted['PUBLIC_NAM'].isin(['Lincoln & Sedgwick']).any()

# Print the result
print("Does route exist in the 'PUBLIC_NAM' column:", exists)

Does route exist in the 'PUBLIC_NAM' column: False


In [18]:
#Looks like these two datasets don't have common routes. 




# Counting unique routes in 'ROUTESSTPG' column of bus_stops_converted DataFrame
unique_routes_bus_stops = bus_stops_converted['ROUTESSTPG'].nunique()

# Counting unique routes in 'route' column of bus_df DataFrame
unique_routes_bus = bus_df['route'].nunique()

# Print the results
print("Number of unique routes in 'ROUTESSTPG' column of bus_stops_converted:", unique_routes_bus_stops)
print("Number of unique routes in 'route' column of bus_df:", unique_routes_bus)


Number of unique routes in 'ROUTESSTPG' column of bus_stops_converted: 516
Number of unique routes in 'route' column of bus_df: 184
